# Estimación de parámetros en redes Bayesianas

<img style="float: right; margin: 0px 0px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/d/d5/Hey_Machine_Learning_Logo.png" width="400px" height="400px" />

> Hasta este momento hemos supuesto que ya conocemos nuestro modelo gráfico (incluyendo estructura y parámetros). Sin embargo, como deben sospechar, este no es el caso en problemas reales.
>
> Una manera de obtener un modelo gráfico es extraer tanto la red como los parámetros con la ayuda de un experto. Esta es una tarea no menor, y puede requerir muchísimo trabajo (semanas, quizá meses, dependiendo del tamaño de la red). Además, el acceso exhaustivo a un experto, puede no ser una suposición plausible.
>
> Por otra parte, es una suposición más común tener acceso a un conjunto de datos generados por la distribución (situación) que queremos modelar. Así, en esta clase nos enfocaremos en construir un modelo a partir de un conjunto de datos.

> **Objetivos:**
> - Describir el problema de aprendizaje de modelos.
> - Obtener los parámetros de máxima verosimilitud para redes Bayesianas.
> - Obtener los parámetros MAP para redes Bayesianas.

> **Referencias:**
> - Probabilistic Graphical Models: Principles and Techniques, By Daphne Koller and Nir Friedman. Ch. 17.
> - Mastering Probabilistic Graphical Models Using Python, By Ankur Ankan and Abinash Panda. Ch. 5.


<p style="text-align:right;"> Imagen recuperada de: https://upload.wikimedia.org/wikipedia/commons/d/d5/Hey_Machine_Learning_Logo.png.</p>

___

## 1. Introducción al aprendizaje de modelos gráficos

El proceso completo de aprendizaje puede ser resumido con el siguiente diagrama:

In [ ]:
from IPython.display import Image

In [ ]:
Image("figures/pgm_construction.png")

### ¿Por qué quisieramos aprender un modelo gráfico?

#### 1. Para responder preguntas desde una perspectiva probabilística al rededor de nuevas observaciones.

¿Qué optimizaremos? **Verosimilitud de datos de entrenamiento**

$$P(\mathcal{D} | \mathcal{M}).$$

- Si un modelo hace que los datos sean más probables, entonces es plausible que los datos provengan de dicho modelo.

Por otra parte, nos interesa evaluar este modelo sobre datos nuevos, no sobre datos que ya hayamos visto.

- De manera que evaluamos el modelo sobre un conjunto separado de pruebas: **Verosimilitud de datos de prueba**
  
  $$P(\mathcal{D}' | \mathcal{M}).$$

#### 2. Para realizar una tarea de predicción específica sobre nuevas instancias.

En este sentido, queremos predecir variables objetivo $\bar{y}$ a partir de ciertas variables observadas $\bar{x}$.

¿Qué optimizamos? Un objetivo especializado: **recall, f1, ...**

- Sin embargo, desde un punto de vista matemático y algorítmico, nos conviene optimizar la verosimilitud de datos de entrenamiento:
  
  $$P(\mathcal{D} | \mathcal{M}) = \prod_{m} P(\bar{y}[m] | \bar{x}[m] : \mathcal{M}).$$
  
Finalmente, el modelo debe ser evaluado sobre los **datos de prueba** y el **objetivo original especializado**.

#### 3. Para descubrir conocimiento acerca de $\mathcal{M}^*$.

En este caso, quisiéramos:

- Entender dependencias directas entre variables (arcos).
- Aprender la direccionalidad de los arcos.
  
¿Qué optimizaremos? **Verosimilitud de datos de entrenamiento**

- Muy conveniente matemáticamente, pero...
- Mala representación de precisión estructural.

### ¿Cómo prevenimos el overfitting?

1. Si seleccionamos el modelo $\mathcal{M}$ optimizando de acuerdo a la verosimilitud de entrenamiento, es bastante probable que caigamos en sobreajustar al ruido.

2. En parámetros:
   - Los parámetros se ajustarán a la incertidumbre de los datos de entrenamiento.
   - Solución: usar regularización, o equivalentemente, previas sobre los parámetros.

3. En la estructura:
   - Verosimilitud de entrenamiento siempre beneficia a las estructuras más complejas.
   - Solución: penalizar la complejidad del modelo.

___
## 2. Parámetros de máxima verosimilitud en redes Bayesianas

**Recordar parámetros de máxima verosimilitud en distribuciones multinomiales**

Apliquemos el principio de máxima verosimilitud para estimar los parámetros en una red Bayesiana.

Supongamos que tenemos un conjunto de muestras iid de la red Bayesiana $X \to Y$:

$$\mathcal{D} = \{(x[1], y[1]), (x[2], y[2]), \dots, (x[M], y[M])\}$$

Podemos dar una interpretación a estas muestras:

In [ ]:
Image("figures/iid_bn.png")

Entonces, tenemos los parámetros de la cpd de $X$, que denotamos como $\theta_X$, y los parámetros de la cpd de $Y$, que denotamos como $\theta_{Y|X}$. Denotaremos a todos los parámetros como $\theta$.

La función de verosimilitud es (<font color = red>ver en el pizarrón</font>):

<!--
$$
\begin{align}
\mathcal{L}(\Theta : \mathcal{D}) & = \prod_{d=1}^{M} P(x[d], y[d] | \Theta) \\
                                  & = \prod_{d=1}^{M} P(x[d] | \Theta) P(y[d] | x[d], \Theta) \\
                                  & = \left(\prod_{d=1}^{M} P(x[d] | \Theta)\right) \left(\prod_{d=1}^{M} P(y[d] | x[d], \Theta)\right) \\
                                  & = \underbrace{\left(\prod_{d=1}^{M} P(x[d] | \theta_X)\right)}_{\text{local likelihood}} \underbrace{\left(\prod_{d=1}^{M} P(y[d] | x[d], \theta_{Y|X})\right)}_{\text{local likelihood}}
\end{align}
$$
-->

Esto se puede generalizar a redes Bayesianas en general:

<!--
$$
\begin{align}
\mathcal{L}(\Theta : \mathcal{D}) & = \prod_{d=1}^{M} P(\bar{x}[d] | \Theta) \\
                                  & = \prod_{d=1}^{M} \prod_{i=1}^{n} P(x_i[d] | \mathrm{Pa}x_i[d], \Theta)  \\
                                  & = \prod_{i=1}^{n} \underbrace{\prod_{d=1}^{M} P(x_i[d] | \mathrm{Pa}x_i[d], \theta_{X_i|\mathrm{Pa}X_i})}_{\text{local likelihood } \mathcal{L}_i(\theta_{X_i|\mathrm{Pa}X_i} : \mathcal{D})}  \\
                                  & = \prod_{i=1}^{n} \mathcal{L}_i(\theta_{X_i|\mathrm{Pa}X_i} : \mathcal{D})
\end{align}
$$
-->

Entonces, si los parámetros $\theta_{X_i|\mathrm{Pa}X_i}$ son disjuntos, entonces, las estimaciones de máxima verosimilitud se pueden llevar a cabo por separado.

### Estimadores de máxima verosimilitud para CPDs tabulares

Usando lo anterior y considerando $\bar{U}=\mathrm{Pa}X$, la verosimilitud de los datos es:

$$
\begin{align}
\prod_{d=1}^{M} P(x[d] | \bar{u}[d], \theta_{X|\bar{U}}) & = \prod_{x \in \mathrm{Val}(X),\bar{u} \in \mathrm{Val}(U)} \prod_{d: x[d]=x, \bar{u}[d]=\bar{u}} \underbrace{P(x[d] | \bar{u}[d], \theta_{X|\bar{U}})}_{\theta_{x|\bar{u}}} \\
& = \prod_{x \in \mathrm{Val}(X),\bar{u} \in \mathrm{Val}(U)} \theta_{x | \bar{u}}^{M(x,\bar{u})},
\end{align}
$$

donde $M(x,\bar{u})$ es el número de veces que la muestra $d$ es consistente con $x, \bar{u}$.

Aplicando el mismo principio que con distribuciones multinomiales:

$$\hat{\theta}_{x|\bar{u}} = \frac{M(x,\bar{u})}{\sum_{x'} M(x',\bar{u})} = \frac{M(x,\bar{u})}{M(\bar{u})},$$

el estimador de máxima verosimilitud corresponde a la cantidad de muestras consistentes con $x,\bar{u}$ entre el la cantidad de muestras consistentes con $\bar{u}$.

### Ejemplo: Clasificador Naive Bayes para dataset de vino

In [ ]:
# Importar BayesianModel, pyplot, pandas, networkx, train_test_split


In [ ]:
# Lectura de datos


In [ ]:
# Quantización de variables numéricas


In [ ]:
# Red Bayesiana


In [ ]:
# Train y test


In [ ]:
# Entrenamos


In [ ]:
# CPDs estimadas


In [ ]:
# Evaluación


### Fragmentación y overfitting

Tenemos que:

$$\hat{\theta}_{x|\bar{u}} = \frac{M(x,\bar{u})}{\sum_{x'} M(x',\bar{u})} = \frac{M(x,\bar{u})}{M(\bar{u})}.$$

- Notamos que el número de posibles asignaciones para $x, \bar{u}$ crece exponencialmente con el tamaño de $|\bar{U}|$.

- De forma que, si $X$ tiene muchos nodos padres, habrá pocas instancias de $x, \bar{u}$.

- Esto nos conllevará a estimadores muy pobres de $\hat{\theta}_{x|\bar{u}}$

Este fenómeno se llama **fragmentación**.

Por tanto, cuando el conjunto de datos es limitado, es común obtener mejor generalización usando estructuras más simples **incluso cuando no son correctas**.

___
## 3. Parámetros MAP en redes Bayesianas

El problema principal de máxima verosimilitud es su tendencia al overfitting en escenarios con datos limitados.

- Supongamos que tiramos una moneda 10 veces, y resulta que cae $x^1$ 7 de las 10 tiradas. Entonces,

  $$\hat{\theta}_{MLE} = 0.7$$
  
- Supongamos que tiramos una moneda 1000 veces, y resulta que cae $x^1$ 700 de las 1000 tiradas. Entonces,

  $$\hat{\theta}_{MLE} = 0.7$$

El estimador del segundo escenario es mucho más plausible que en el primero. Sin embargo, el principio de máxima verosimilitud no distingue entre ambos.

**Paradigma Bayesiano:** Si tenemos incertidumbre de algo, entonces deberíamos tratar ese algo como variable aleatoria, sobre la cual tenemos una distribución, la cual actualizamos en el tiempo en tanto adquirimos más datos.

Bajo este paradigma, los parámetros $\theta$ serán variables aleatorias.

- Dado $\theta$, los tiros son independientes.

- Si  $\theta$ es desconocido, entonces los tiros no son independientes:
  - Cada tiro nos dice algo acerca de $\theta$.

#### En una distribución multinomial

Probabilidad conjunta:

$$
\begin{align}
P(x[1], \dots, x[M], \theta) & = P(x[1], \dots, x[M] | \theta) P(\theta) \\
                             & = P(\theta) \underbrace{\prod_{d=1}^{M} P(x[d] | \theta)}_{\text{likelihood function}} \\
                             & = P(\theta)\prod_{i=1}^{k} \theta_k^{M_i}.
\end{align}
$$

Distribución posterior:

$$
\begin{align}
P(\theta | x[1], \dots, x[M]) & = \frac{P(x[1], \dots, x[M], \theta)}{P(x[1], \dots, x[M])} \\
                              & \propto P(x[1], \dots, x[M], \theta) \\
                              & = P(\theta)\prod_{i=1}^{k} \theta_k^{M_i}
\end{align}
$$

### Previa: Distribution de Dirichlet (generalización de la distribución Beta)

Si $X$ es una distribución multinomial sobre $k$ valores, una buena suposición para la distribución de los parámetros $\bar{\theta} = [\theta_1, \dots, \theta_k]$ es la [**distribución de Dirichlet**](https://en.wikipedia.org/wiki/Dirichlet_distribution).

$\bar{\theta} \sim \mathrm{Dir}(\bar{\alpha})$, donde $\bar{\alpha}=[\alpha_1, \dots, \alpha_k]$,  si

$$P(\bar{\theta}; \bar{\alpha}) = \frac{1}{Z} \prod_{i=1}^{k} \theta_i^{\alpha_i - 1},$$

donde $\sum_{i=1}^{k}\theta_i=1$ y,

$$Z = \frac{\prod_{i=1}^{k} \Gamma(\alpha_i)}{\Gamma(\sum_{i=1}^{k}\alpha_i)},$$

donde $\Gamma(x)= \int_{0}^{\infty}t^{x-1}e^{-t} \mathrm{d}t$ es una extensión continua del factorial.

1. Media
  
   $$
   \mathbb{E}[\theta_k] = \frac{\alpha_k}{\sum_{j=1}^{k}\alpha_j}
   $$
   
2. Moda
     
   $$
   \text{Mode}[\theta_k] = \frac{\alpha_k - 1}{\sum_{j=1}^{k}\alpha_j - K}
   $$

3. Varianza
   
   $$
   \text{Var}[\theta_k] = \frac{\alpha_k (\alpha_0 - \alpha_k)}{\alpha_0^2 (\alpha_0 + 1)}
   $$
   
4. Covarianza
   
   $$
   \text{Cov}[\theta_k, \theta_j] = \frac{- \alpha_k \alpha_j}{\alpha_0^2 (\alpha_0 + 1)}, \qquad \text{para } j\neq k
   $$

- Intuitivamente, los hiperparámetros $\alpha_i$ corresponden al número de muestras con valor $i$ que hemos visto anteriormente.

In [ ]:
# Distribución beta en scipy.stats


In [ ]:
# Instanciamos varias distribuciones beta


In [ ]:
# Graficamos


### Con la previa de Dirichlet, ¿Cómo es la posterior?

- La función de verosimilitud es: $P(\mathcal{D} | \bar{\theta}) = \prod_{i=1}^{k} \theta_i^{M_i}$.

- La previa es: $P(\bar{\theta}) \propto \prod_{i=1}^{k} \theta_i^{\alpha_i - 1}$

Entonces,

$$
P(\bar{\theta} | \mathcal{D}) \propto \prod_{i=1}^{k} \theta_i^{\alpha_i + M_i - 1}.
$$

Esto es, una distribución de Dirichlet $(\bar{\theta} | \mathcal{D}) \sim \mathrm{Dir}(\alpha_1 + M_1, \dots, \alpha_k + M_k)$.

### En este sentido ...

Antes de ver los datos:

$$P(X=x^i | \theta) = \frac{\alpha_i}{\sum_{j=1}^{k}\alpha_j} = \frac{\alpha_i}{\alpha}.$$

Después de ver los datos:

$$P(X[M+1]=x^i | \theta, x[1], \dots, x[M]) = \frac{\alpha_i + M_i}{\alpha + M}.$$


- EL término $\alpha=\sum_{j=1}^{k}\alpha_j$ se conoce como **equivalent sample size**.
  - Un $\alpha$ más grande significa que tenemos más certeza de nuestra previa.

### ¿Cuál es el efecto?

In [ ]:
# Generate random numbers form a Bernoulli distribution
theta_true = 0.7


In [ ]:
# Obtain MLE estimation at each step


In [ ]:
# Obtain Bayesian estimation for different values of alpha


In [ ]:
# Plot


- Estimadores MAP son menos sensibles al ruido en los datos.

- Estimador MAP $\to$ estimador de máxima verosimilitud cuando el número de muestras $\to \infty$.

**Así como extendimos las ideas de máxima verosimilitud de una distribución multinomial a redes Bayesianas, similarmente sucede con estimadores MAP.**

Todos los detalles en *Probabilistic Graphical Models: Principles and Techniques, By Daphne Koller and Nir Friedman. Ch. 17.*

### Ejemplo: Clasificador Naive Bayes para dataset de vino

In [ ]:
# Definimos de nuevo el modelo


In [ ]:
# Instanciamos estimador bayesiano


In [ ]:
# Estimar CPDs con previa de Dirichlet


In [ ]:
# Añadimos CPDs


In [ ]:
# Evaluación


# Anuncios

## 1. Quiz.

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>